In [1]:
import sys, os
p = os.path.abspath('..')
# p = p+r'\config'
sys.path.insert(1, p)

In [2]:
import pandas as pd
import re
import numpy as np


In [3]:
path = './result'
file_list = os.listdir(path)

In [4]:
def calc_acc_condition(llm_model, few_shot_n, test_n, q_src_yn, p_ver):
    opt_file = [x for x in file_list if x.startswith(f'{llm_model}_result_{few_shot_n}_{test_n}_{q_src_yn}')]
    opt_file = [x for x in opt_file if x.endswith(f'_{p_ver}.csv')]

    if len(opt_file)>0 : 
        df = pd.DataFrame()

        for f in opt_file:
            tmp = pd.read_csv(f'./result/{f}', index_col =0)
            df = pd.concat([df, tmp], axis =0)
        
        df['gold'] = df['answer_encode'].apply(lambda x : re.sub(r'[^0-9]', '', x))
        df['o_result'] = df['result'].apply(lambda x : re.sub(r'[^0-9]', '', x))
        df['equal_yn'] = np.where(df['gold']==df['o_result'], 1, 0)
        acc = (df['equal_yn'].sum()/df.shape[0])*100
        print(f'{llm_model}_result_{few_shot_n}_{test_n}_{q_src_yn} : ', acc)


In [5]:
calc_acc_condition('c' , 2, 10, 'N', 'sys_prompt1')
calc_acc_condition('c' , 2, 10, 'Y', 'sys_prompt1')
calc_acc_condition('c' , 3, 10, 'N', 'sys_prompt1')
calc_acc_condition('c' , 3, 10, 'Y', 'sys_prompt1')



calc_acc_condition('l' , 1, 10, 'N', 'sys_prompt1')
calc_acc_condition('l' , 1, 10, 'Y', 'sys_prompt1')
calc_acc_condition('l' , 2, 10, 'N', 'sys_prompt1')
calc_acc_condition('l' , 2, 10, 'Y', 'sys_prompt1')
calc_acc_condition('l' , 3, 10, 'N', 'sys_prompt1')
calc_acc_condition('l' , 3, 10, 'Y', 'sys_prompt1')
calc_acc_condition('l' , 4, 10, 'N', 'sys_prompt1')
calc_acc_condition('l' , 4, 10, 'Y', 'sys_prompt1')


l_result_1_10_N :  59.0
l_result_1_10_Y :  66.0
l_result_2_10_N :  59.0
l_result_2_10_Y :  56.00000000000001
l_result_3_10_N :  51.0
l_result_3_10_Y :  56.00000000000001


In [6]:
calc_acc_condition('c' , 2, 10, 'N', 'sys_prompt2')
calc_acc_condition('c' , 2, 10, 'Y', 'sys_prompt2')
calc_acc_condition('c' , 3, 10, 'N', 'sys_prompt2')
calc_acc_condition('c' , 3, 10, 'Y', 'sys_prompt2')
calc_acc_condition('l' , 1, 10, 'N', 'sys_prompt2')
calc_acc_condition('l' , 1, 10, 'Y', 'sys_prompt2')
calc_acc_condition('l' , 2, 10, 'N', 'sys_prompt2')
calc_acc_condition('l' , 2, 10, 'Y', 'sys_prompt2')
calc_acc_condition('l' , 3, 10, 'N', 'sys_prompt2')
calc_acc_condition('l' , 3, 10, 'Y', 'sys_prompt2')
calc_acc_condition('l' , 4, 10, 'N', 'sys_prompt2')
calc_acc_condition('l' , 4, 10, 'Y', 'sys_prompt2')


l_result_1_10_N :  64.0
l_result_1_10_Y :  60.0
l_result_2_10_N :  63.0
l_result_2_10_Y :  66.0
l_result_3_10_N :  64.0
l_result_3_10_Y :  69.0
